In [1]:
using LightXML

In [5]:
using FileIO

In [2]:
"""
Create and return XML string with xyzt
"""
function generatexml(SizeX::Integer, SizeY::Integer, SizeZ::Integer, SizeT::Integer)
    tiffXML = XMLDocument();
    OME = create_root(tiffXML, "OME");
    xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" 
    xmlns_xsi="http://www.w3.org/2001/XMLSchema-instance"
    Creator="OME Bio-Formats 5.2.2" 
    UUID="urn:uuid:a5ae8c1b-ac04-4544-97c1-bbdd0bdf8629" # use fixed value now
    xsi_schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd"
    OME_attributes = Dict("xmlns"=>xmlns, "xmlns:xsi"=>xmlns_xsi, "Creator"=>Creator, "UUID"=>UUID, "xsi:schemaLocation"=>xsi_schemaLocation)
    set_attributes(OME, OME_attributes)
    
    name = "test.tiff"
    Image= new_child(OME, "Image")
    set_attributes(Image, Dict("ID"=>"Image:0", "Name"=>name))
    
    Pixels = new_child(Image, "Pixels");
    SizeC=1; 
    #SizeT=10;  SizeX=1300; SizeY=1900; SizeZ=2;
    PhysicalSizeX=PhysicalSizeY=0.108; PhysicalSizeZ=0.5;
    TimeIncrement=600;
    Pixels_attributes = Dict("ID"=>"Pixels:0","Type"=>"uint16", "BigEndian"=>"false",
    "DimensionOrder"=>"XYZCT", "SizeC"=>SizeC,"SizeX"=>SizeX, "SizeY"=>SizeY, "SizeZ"=>SizeZ, "SizeT"=>SizeT,
    "PhysicalSizeX"=>PhysicalSizeX, "PhysicalSizeY"=>PhysicalSizeY, "PhysicalSizeZ"=>PhysicalSizeZ,
    "PhysicalSizeXUnit"=>"µm", "PhysicalSizeYUnit"=>"µm", "PhysicalSizeZUnit"=>"µm",
    "TimeIncrement"=>TimeIncrement, "TimeIncrementUnit"=>"s");
    set_attributes(Pixels, Pixels_attributes)
    
    Channel_ = new_child(Pixels, "Channel");
    set_attributes(Channel_, Dict("ID"=>"Channel:0:0", "SamplesPerPixel"=>"1"));
    LightPath =new_child(Channel_, "LightPath");
    
    """
    for i in 1:SizeC*SizeZ*SizeT
        new_child(Pixels, "TiffData")
    end
    """
    for c in 0:SizeC-1
        for t in 0:SizeT-1
            for z in 0:SizeZ-1
                TiffData = new_child(Pixels, "TiffData")
                TiffData_attributes = Dict("FirstC"=>c, "FirstT"=>t, "FirstZ"=>z, "IFD"=>t*SizeZ+z,
                    "PlaneCount"=>1)
                set_attributes(TiffData, TiffData_attributes)
            end
        end
    end
    
    tiffXML;
end

"""
Embed OME-XML to tiff with XYZT informance
"""
function embedxml(SizeX::Integer, SizeY::Integer, SizeZ::Integer, SizeT::Integer, img_name::String)
    tiffxml = generatexml(SizeX, SizeY, SizeZ, SizeT);
    print("Embedding OME-XML")
    run(`/home/hf/Bin/bftools/tiffcomment -set $tiffxml $img_name`);
    nothing;
end

embedxml

In [3]:
tiffxml = generatexml(512, 512, 20, 138);
#run(`/home/hf/Bin/bftools/tiffcomment -set $tiffxml path1-1-z.tif`);

In [8]:
save_file(tiffxml, "new.xml");

In [28]:
run(pipeline(`echo $tiffxml`,`/home/hf/Bin/bftools/xmlvalid  `));

Parsing schema path
http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd
Validating <stdin>
No validation errors found.


In [ ]:
"d16s1_2_otsu.ome.tiff"

In [13]:
run(`exiftool -ImageDescription\<=new.xml "d16s1_2_otsu.ome.tiff" -overwrite_original`)

    1 image files updated


Process(`exiftool '-ImageDescription<=new.xml' d16s1_2_otsu.ome.tiff -overwrite_original`, ProcessExited(0))

In [16]:
print(tiffxml)

<?xml version="1.0" encoding="utf-8"?>
<OME xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:a5ae8c1b-ac04-4544-97c1-bbdd0bdf8629" Creator="OME Bio-Formats 5.2.2" xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <Image Name="test.tiff" ID="Image:0">
    <Pixels PhysicalSizeXUnit="µm" PhysicalSizeYUnit="µm" SizeC="1" DimensionOrder="XYZCT" SizeZ="20" BigEndian="false" PhysicalSizeX="0.108" SizeT="13" ID="Pixels:0" TimeIncrementUnit="s" TimeIncrement="600" PhysicalSizeZUnit="µm" SizeX="512" PhysicalSizeZ="0.5" Type="uint16" SizeY="512" PhysicalSizeY="0.108">
      <Channel SamplesPerPixel="1" ID="Channel:0:0">
        <LightPath/>
      </Channel>
      <TiffData FirstT="0" IFD="0" FirstC="0" PlaneCount="1" FirstZ="0"/>
      <TiffData FirstT="0" IFD="1" FirstC="0" PlaneCount="1" FirstZ="1"/>
      <TiffData FirstT="0" IFD="2" Fi